<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="ITM">
</head>

<img src="https://maua.br/images/selo-60-anos-maua.svg" width=300, align="right">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->




# **P1 - RA ÍMPAR**

In [ ]:
#@markdown Nome completo e RA (*\<nome\>, \<RA\>*)
Aluno = 'Daniel Henrique, 1115665' #@param {type:"string"}

In [ ]:
import pandas as pd
import numpy as np
path = 'https://github.com/Rogerio-mack/work/raw/main/data/'

# **Case**: Airbnb

Neste case, analise dados sobre imóveis alugados pela plataforma Airbnb em NYC no ano de 2019.


In [ ]:
df = pd.read_csv(path + 'AB_NYC_2019.csv')
print(len(df))
df.head()

804


id                                            name  host_id  \
0   20913              Charming 1 bed GR8 WBurg LOCATION!    79402   
1   25235               Large 2 Bedroom Great for Groups!    87773   
2   76761  Eveland the Place to Stay & Enjoy a 5-⭐️ 2bdrm   118971   
3  135465             Garden apartment close to Manhattan   663879   
4  169002                Modern Space in Charming Pre-war   805344   

     host_name neighbourhood_group       neighbourhood  latitude  longitude  \
0   Christiana            Brooklyn        Williamsburg  40.70984  -73.95775   
1       Shelly            Brooklyn  Bedford-Stuyvesant  40.68373  -73.92377   
2       Evelyn            Brooklyn         South Slope  40.66552  -73.99019   
3  Christopher            Brooklyn         Fort Greene  40.68626  -73.97598   
4         Alec           Manhattan              Harlem  40.82411  -73.94934   

         room_type  price  minimum_nights  number_of_reviews last_review  \
0  Entire home/apt    100               5                168  2018-07-22   
1  Entire home/apt    125              90                162  2019-06-28   
2  Entire home/apt    169               2                398  2019-06-28   
3  Entire home/apt    170               3                 86  2019-06-20   
4     Private room     65               2                 41  2019-06-16   

   reviews_per_month  calculated_host_listings_count  availability_365  
0               1.57                               1                 0  
1               1.46                               2               137  
2               3.97                               3               182  
3               0.91                               2               286  
4               0.43                               2                59

# Q1. Proprietários com mais imóveis na plataforma

Obtenha os 3 proprietários ("hosts") com mais imóveis sendo anunciados na plataforma, e a respectiva quantidade de imóveis anunciados.


In [ ]:
df_q1 = df.groupby('host_name').id.count().sort_values(ascending=False).head(3)
df_q1

host_name
Sonder (NYC)    7
Mark            7
David           6
Name: id, dtype: int64

# Q2B. Preço nos bairros com mais imóveis

 Qual o preço médio dos imóveis, considerando somente os preços dos imóveis cujos bairros (`neighbourhood`) tenham mais de 20 imóveis sendo anunciados?  


In [ ]:
# pegar os bairros com mais de 20 imóveis
df_q2 = df.groupby('neighbourhood').id.count().sort_values(ascending=False)
# filtrar apenas os com mais de 20
df_q2 = df_q2[df_q2 > 20]
# pegar os preços desses bairros
df_q2 = df[df.neighbourhood.isin(df_q2.index)]
# calcular o preço médio
preco_medio = df_q2.price.mean()
print(f'O preço médio dos imóveis em bairros com mais de 20 anúncios é: USD {preco_medio:.2f}')

O preço médio dos imóveis em bairros com mais de 20 anúncios é: USD 151.42


# Q3B.  Brooklyn *outliers*

Qual o percentual de imóveis de Manhattan com preços estatisticamente discrepantes (*outliers*) com relação aos preços de todos os imóveis?



In [ ]:
Q1 = df['price'].quantile(0.25)
Q3 = df['price'].quantile(0.75)
# filtrar apenas manhattan
df_brooklyn = df[df.neighbourhood_group == 'Brooklyn']
# pegar os outliers
outliers = df_brooklyn[(df_brooklyn['price'] < Q1 - 1.5 * (Q3 - Q1)) | (df_brooklyn['price'] > Q3 + 1.5 * (Q3 - Q1))]
perc_Brooklyn = (outliers.shape[0] / df_brooklyn.shape[0]) * 100
print( f'Percentuais de outliers de preços, Brooklyn: {perc_Brooklyn:.2f} %')


Percentuais de outliers de preços, Brooklyn: 3.94 %


# Q4B. Scores

Nesta tarefa você deve atribuir a cada imóvel um valor de *score* (`df['score']`) baseado nas seguintes regras:

1. Imóveis com `availability_365` > 90 recebem +10 pontos enquanto imóveis com `availability_365` < 5 recebem -5 pontos;
2. Imóveis com o número de reviews >= 10 recebem +5 pontos para cada 10 reviews (por ex. se número de reviews = 20, são atribuídos +10 pontos). Se número de reviews = 0 o imóvel recebe -5 pontos;
3. Imóveis com preço abaixo da média de seu `neighbourhood`	recebem +5 pontos.

Implemente empregando `apply()` ou um laço de programa, como preferir.

In [ ]:
...
df['score'] = 0
df['score'] = df.apply(lambda x: 10 if x['availability_365'] > 90 else x['score'], axis=1)
df['score'] = df.apply(lambda x: -5 if x['availability_365'] < 5 else x['score'], axis=1)
df['score'] = df.apply(lambda x: x['score'] + 5 if x['number_of_reviews'] >= 10 else x['score'], axis=1)
df['score'] = df.apply(lambda x: x['score'] - 5 if x['number_of_reviews'] == 0 else x['score'], axis=1)
df['score'] = df.apply(lambda x: x['score'] + 5 if x['price'] < df[df['neighbourhood'] == x['neighbourhood']]['price'].mean() else x['score'], axis=1)


print(f'A soma dos valores de score é: ', df['score'].sum())


A soma dos valores de score é:  4835


# Q5. Enriquecendo os dados

Enriqueça os dados dos imóveis com os valores dos impostos e multas municipais. Cacule a média de impostos e multas de cada grupo de vizinhaça, `neighbourhood_group`. Note que existem imóveis sem impostos e sem multas, esses são imóveis isentos, mas devem ser considerados nos cálculos de média (por exemplo, você pode considerar esses imóveis com valor 0).

In [ ]:
df_taxes = pd.read_csv(path + 'AB_NYC_2019_taxes.csv')
df_taxes = df_taxes.fillna(0)
df_taxes

id   type   value
0     35907127  taxes  1315.0
1       804815  taxes     0.0
2     22328585  taxes   900.0
3     27876600  taxes   869.0
4     34976037  taxes  1629.0
...        ...    ...     ...
1523  34284316  fines     0.0
1524  20325804  fines     0.0
1525  25872986  fines     0.0
1526  19365218  fines   906.0
1527   4235843  fines     0.0

[1528 rows x 3 columns]

In [ ]:
df_q5 = pd.merge(df, df_taxes, on='id', how='left')
df_q5['fines'] = 0
df_q5['taxes'] = 0
df_q5['fines'] = df_q5.apply(lambda x: x['value'] if x['type'] == 'fines' else 0, axis=1)
df_q5['taxes'] = df_q5.apply(lambda x: x['value'] if x['type'] == 'taxes' else 0, axis=1)
df_q5 = df_q5.groupby('neighbourhood_group').agg({'fines': 'mean', 'taxes': 'mean'})
df_q5





fines       taxes
neighbourhood_group                        
Bronx                 61.615385  414.615385
Brooklyn             151.558870  413.643642
Manhattan            154.436288  435.804709
Queens               160.356688  399.522293
Staten Island        268.307692  534.230769

# Q6. Quebras

Construa no `Pandas` um "relatório" (`df_report`) como abaixo. Os valores são ordenados `neighbourhood_group`,`neighbourhood`. Cada "página" contém apenas 30 linhas. A cada "quebra" de `neighbourhood_group` é gerada uma linha com preço médio do `neighbourhood_group` e, então, é iniciada uma nova página.

O item `line_id` é opcional mas deve auxiliá-lo nessa tarefa.

Ao final produza a soma dos preços das páginas abaixo...

neighbourhood_group       price   neighbourhood
0                 Bronx   71.500000             NaN
1              Brooklyn  128.087879             NaN
2             Manhattan  196.157609             NaN
3                Queens   92.227848             NaN
4         Staten Island   89.428571             NaN
..                  ...         ...             ...
804            Brooklyn   80.000000    Williamsburg
805              Queens   60.000000       Sunnyside
806            Brooklyn   70.000000        Bushwick
807            Brooklyn  800.000000  Sheepshead Bay
808           Manhattan   80.000000          Inwood

[809 rows x 3 columns]

In [ ]:
df_report = df.copy()

df_report = df_report.sort_values(by=['neighbourhood_group', 'neighbourhood'])

df_report['line_id'] = range(len(df_report))

page_num = 1
line_count = 0
page_prices = []
current_group = ''

report_data = []

for index, row in df_report.iterrows():
    if row['neighbourhood_group'] != current_group:
        if current_group != '':

            avg_price = df_report[df_report['neighbourhood_group'] == current_group]['price'].mean()
            report_data.append({'neighbourhood_group': current_group, 'avg_price': avg_price, 'line_id': None, 'page': page_num})
            page_prices.append(avg_price)


            if line_count >= 30:
              page_num += 1
              line_count = 0


        current_group = row['neighbourhood_group']
        line_count = 0


    report_data.append({'neighbourhood_group': row['neighbourhood_group'], 'neighbourhood': row['neighbourhood'], 'price': row['price'], 'line_id': row['line_id'], 'page': page_num})
    page_prices.append(row['price'])
    line_count += 1


avg_price = df_report[df_report['neighbourhood_group'] == current_group]['price'].mean()
report_data.append({'neighbourhood_group': current_group, 'avg_price': avg_price, 'line_id': None, 'page': page_num})
page_prices.append(avg_price)

df_report = pd.DataFrame(report_data)
sum_price = df_report['price'].sum()
print(f"Soma dos preços das páginas: {sum_price}")
avg_price = df_report['avg_price'].sum()
print(f"Preço médio das páginas: {avg_price}")
print(df_report)
df_report = df_report[df_report['page'].isin([1,24,25,26,28])]
sum_price = df_report['price'].sum()
print(f"Soma dos preços das páginas: {sum_price}")
avg_price = df_report['avg_price'].sum()
print(f"Preço médio das páginas: {avg_price}")
print(f"Soma de ambos os valores: {sum_price + avg_price}")



Soma dos preços das páginas: 123797.0
Preço médio das páginas: 577.4019070133683
    neighbourhood_group      neighbourhood  price  line_id  page  avg_price
0                 Bronx            Belmont   29.0      0.0     1        NaN
1                 Bronx  Claremont Village   67.0      1.0     1        NaN
2                 Bronx          Concourse   40.0      2.0     1        NaN
3                 Bronx           Edenwald   90.0      3.0     1        NaN
4                 Bronx           Edenwald   80.0      4.0     1        NaN
..                  ...                ...    ...      ...   ...        ...
804       Staten Island        South Beach  100.0    800.0     4        NaN
805       Staten Island         St. George  130.0    801.0     4        NaN
806       Staten Island          Todt Hill  135.0    802.0     4        NaN
807       Staten Island      Tompkinsville   59.0    803.0     4        NaN
808       Staten Island                NaN    NaN      NaN     4  89.428571

[809 r

#### Aqui um *sample* do report.

In [ ]:
df_report.iloc[-10::,:]